In [1]:
import findspark
findspark.init('/home/shashank/spark-2.3.2-bin-hadoop2.7')

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [5]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [7]:
data.createOrReplaceTempView('data')

In [9]:
spark.sql("FROM data SELECT userId , AVG(rating) GROUP BY userId").show()

+------+------------------+
|userId|       avg(rating)|
+------+------------------+
|    28|              1.82|
|    26| 2.204081632653061|
|    27| 1.608695652173913|
|    12|1.8545454545454545|
|    22|2.1607142857142856|
|     1|1.5510204081632653|
|    13|1.5208333333333333|
|     6|1.4385964912280702|
|    16|1.7777777777777777|
|     3|1.8333333333333333|
|    20|1.7659574468085106|
|     5|1.7755102040816326|
|    19| 1.469387755102041|
|    15|1.5416666666666667|
|     9|1.7924528301886793|
|    17|1.9565217391304348|
|     4|1.5636363636363637|
|     8|1.8979591836734695|
|    23|2.1346153846153846|
|     7|1.6296296296296295|
+------+------------------+
only showing top 20 rows



In [10]:
from pyspark.ml.recommendation import ALS

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [13]:
train, test = data.randomSplit([0.8, 0.2])

In [15]:
als = ALS(userCol= 'userId', itemCol= 'movieId', ratingCol= 'rating')

In [16]:
als_model = als.fit(train)

In [17]:
predictions = als_model.transform(test)

In [18]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|     5| 1.6825107|
|     31|   1.0|     4| 1.6147572|
|     31|   3.0|    14| 2.1577373|
|     31|   1.0|     0| 1.2970935|
|     85|   1.0|    13| 1.6835635|
|     65|   1.0|    16|0.65728736|
|     65|   2.0|     3| 0.9968534|
|     65|   1.0|    24| 1.1101536|
|     53|   1.0|     6| 2.9480176|
|     53|   1.0|     9| 2.8942575|
|     53|   1.0|     7|  2.732754|
|     53|   1.0|    25| 1.3755968|
|     78|   1.0|    12|0.70797193|
|     78|   1.0|    22| 1.2872472|
|     78|   1.0|    20|0.81456923|
|     78|   1.0|    19| 0.6572957|
|     78|   1.0|    17|0.72381383|
|     78|   1.0|     8| 1.0063795|
|     34|   1.0|    28| 1.3742796|
|     34|   3.0|     3| 0.9620608|
+-------+------+------+----------+
only showing top 20 rows



In [19]:
evaluate = RegressionEvaluator(predictionCol='prediction', labelCol='rating') #metricname = rmse

In [20]:
results = evaluate.evaluate(predictions)

In [22]:
print("RMSE")
print(results)

RMSE
0.9495967928031895


Not that great! probably because how small the data is.

# Single User

In [23]:
test.createOrReplaceTempView('test')

In [24]:
single_user = spark.sql("FROM test SELECT * WHERE userId == 11")

In [25]:
single_user.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|     37|   2.0|    11|
|     40|   1.0|    11|
|     41|   1.0|    11|
|     43|   2.0|    11|
|     47|   1.0|    11|
|     51|   3.0|    11|
|     79|   5.0|    11|
|     90|   4.0|    11|
+-------+------+------+



In [26]:
recommendations = als_model.transform(single_user)

In [27]:
recommendations.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     47|   1.0|    11| 1.0512478|
|     40|   1.0|    11| 1.6170714|
|     41|   1.0|    11|0.78259563|
|     43|   2.0|    11| 1.5583735|
|     37|   2.0|    11|  2.175357|
|     51|   3.0|    11| 2.1637008|
|     90|   4.0|    11|  2.149203|
|     79|   5.0|    11| 1.9546902|
+-------+------+------+----------+



# Meal Info

In [48]:
data = spark.read.csv('Meal_Info.csv', inferSchema=True, header = True)

In [49]:
data.createOrReplaceTempView('data')

In [50]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)
 |-- mealskew: double (nullable = true)
 |-- meal_name: string (nullable = true)



In [65]:
data = spark.sql("FROM data SELECT movieId, rating, userId, mealskew")
data.show()

+-------+------+------+--------+
|movieId|rating|userId|mealskew|
+-------+------+------+--------+
|      2|   3.0|     0|     2.0|
|      3|   1.0|     0|     3.0|
|      5|   2.0|     0|     5.0|
|      9|   4.0|     0|     9.0|
|     11|   1.0|     0|    11.0|
|     12|   2.0|     0|    12.0|
|     15|   1.0|     0|    15.0|
|     17|   1.0|     0|    17.0|
|     19|   1.0|     0|    19.0|
|     21|   1.0|     0|    21.0|
|     23|   1.0|     0|    23.0|
|     26|   3.0|     0|    26.0|
|     27|   1.0|     0|    27.0|
|     28|   1.0|     0|    28.0|
|     29|   1.0|     0|    29.0|
|     30|   1.0|     0|    30.0|
|     31|   1.0|     0|    31.0|
|     34|   1.0|     0|    null|
|     37|   1.0|     0|    null|
|     41|   2.0|     0|    null|
+-------+------+------+--------+
only showing top 20 rows



In [66]:
from pyspark.ml.feature import VectorAssembler

In [67]:
assembler = VectorAssembler(inputCols=['rating', 'mealskew'], outputCol='features')

In [69]:
new_data = assembler.transform(data)

In [70]:
new_data.createOrReplaceTempView('new_data')

In [71]:
spark.sql("FROM new_data SELECT *").show()

Py4JJavaError: An error occurred while calling o520.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 348.0 failed 1 times, most recent failure: Lost task 0.0 in stage 348.0 (TID 1496, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<rating:double,mealskew:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:836)
	at

In [58]:
als_model = ALS(userCol='userId', itemCol='movieId', seed = 101, ratingCol='features')

In [59]:
pipeline = Pipeline(stages=[meal_indexer, meal_encoder, assembler, als_model])

In [60]:
train, test = data.randomSplit([0.8,0.2])

In [61]:
fit_model = pipeline.fit(train)

IllegalArgumentException: 'requirement failed: Column features must be of type NumericType but was actually of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7.'